In [1]:
!ls

00_DevelopModel.ipynb			 conda_dependencies.yml
01_DevelopModelDriver.ipynb		 config
02_BuildImage.ipynb			 docker.runconfig
220px-Lynx_lynx_poing (1).jpg		 driver.py
220px-Lynx_lynx_poing.jpg		 environment.yml
AML_00_configuration.ipynb		 local.runconfig
AML_00_configuration_v0.1.50.ipynb	 project.json
AML_00_configuration_v0.1.65.ipynb	 __pycache__
AML_01_trainlocal-docker.ipynb		 pytorch_resnet152
AML_01_trainlocal.ipynb			 pytorch_resnet152.tar.gz
AML_01_trainlocal_v0.1.50.ipynb		 resnet152-b121ed2d.pth
AML_01_trainlocal_v0.1.65.ipynb		 resnet152.py
AML_02_AKS_deployment.ipynb		 score.py
AML_02_AKS_deployment_v0.1.50.ipynb	 synset.txt
AML_02_AKS_deployment_v0.1.65.ipynb	 testing_utilities.py
AML_02_Develop_init_run_functions.ipynb  train.py
aml_config


In [21]:
from azureml.core.image import ContainerImage

In [15]:
%%writefile deploy_environment.yml
channels:
  - conda-forge
  - pytorch
dependencies:
  - python=3.6
  - pytorch 
  - pip:
    - python-dotenv==0.9.0
    - Pillow==5.2.0
    - wget==3.2
    - matplotlib==2.2.2
    - toolz==0.9.0
    - tqdm==4.23.4
    - torchvision
    - azure
    - azureml-sdk==0.1.68

Writing aml_environment.yml


In [22]:
image_config = ContainerImage.image_configuration(execution_script = "score.py",
                                                  runtime = "python",
                                                  conda_file = "aml_environment.yml",
                                                  description = "Image for AKS Deployment Tutorial",
                                                  tags = {"name":"AKS","project":"AML"}, 
                                                  dependencies = ["synset.txt"],
                                                  enable_gpu = True
                                                 )

In [25]:
from azureml.core import Workspace

In [26]:
my_workspace = Workspace.from_config()
my_workspace.get_details()

Found the config file in: /home/mat/repos/AKSDeploymentTutorial_AML/Pytorch/aml_config/config.json


{'id': '/subscriptions/edf507a2-6235-46c5-b560-fd463ba2e771/resourceGroups/amlakspytorchrg/providers/Microsoft.MachineLearningServices/workspaces/pytorch_workspace',
 'name': 'pytorch_workspace',
 'location': 'eastus2',
 'type': 'Microsoft.MachineLearningServices/workspaces',
 'description': '',
 'friendlyName': 'pytorch_workspace',
 'containerRegistry': '/subscriptions/edf507a2-6235-46c5-b560-fd463ba2e771/resourcegroups/amlakspytorchrg/providers/microsoft.containerregistry/registries/pytorchwacrrmwuvfch',
 'keyVault': '/subscriptions/edf507a2-6235-46c5-b560-fd463ba2e771/resourcegroups/amlakspytorchrg/providers/microsoft.keyvault/vaults/pytorchwkeyvaultgzktbsnx',
 'applicationInsights': '/subscriptions/edf507a2-6235-46c5-b560-fd463ba2e771/resourcegroups/amlakspytorchrg/providers/microsoft.insights/components/pytorchwinsightslwbyjuau',
 'batchaiWorkspace': '/subscriptions/edf507a2-6235-46c5-b560-fd463ba2e771/resourcegroups/amlakspytorchrg/providers/microsoft.batchai/workspaces/pytorchwb

In [33]:
workspace_models = my_workspace.models()

In [34]:
workspace_models

{'pytorch_resnet152': <azureml.core.model.Model at 0x7fb5cb8eb7b8>}

In [35]:
image = ContainerImage.create(name = "image1",
                              # this is the model object
                              models = [workspace_models['pytorch_resnet152']],                              
                              image_config = image_config,
                              workspace = my_workspace)

image.wait_for_creation(show_output = True)

Creating image
Running..........................................................................................................................................................
SucceededImage creation operation finished for image image1:1, operation "Succeeded"


In [36]:
image.image_location

'pytorchwacrrmwuvfch.azurecr.io/image1:1'

In [40]:
IMAGEURL = "https://upload.wikimedia.org/wikipedia/commons/thumb/6/68/Lynx_lynx_poing.jpg/220px-Lynx_lynx_poing.jpg"

In [41]:
from testing_utilities import img_url_to_json

In [42]:
jsonimg = img_url_to_json(IMAGEURL)

In [44]:
type(image)

azureml.core.image.container.ContainerImage

In [43]:
image.run(jsonimg)

WebserviceException: Failed to create Docker client: Error while fetching server API version: ('Connection aborted.', PermissionError(13, 'Permission denied'))

In [ ]:
my_workspace.